<a href="https://colab.research.google.com/github/younghoonNa/Parallel_Computer_Architecture/blob/main/CUDA/Multiprocessing_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Example1

## Example1

In [ ]:
%%writefile CUDA_practice.cu

#include <stdio.h>

__device__ void hiDeviceFunction(void) // GPU에서만 돌아가며, GPU에서만 Qualify 된다..
{
  printf("Hello! This is in hiDeviceFunciton. \n");
}

__global__ void helloCUDA(void) // Local + GPU
{
  printf("Hello thread %d\n", threadIdx.x); // <<< 에 의해서 만들어지는 threadIdx
  hiDeviceFunction();
}

int main()
{
  helloCUDA<<<1, 1>>>(); // <<< -> block 하나를만들어라, >>> 1개의 Thread를 만들어라.

  //Asynchronous에 의해 Function이 호출 되기 전에 종료가 되어버림. 
  cudaDeviceSynchronize(); //cudaDeviceReset 도 가능

  return 0;
}

Writing CUDA_practice.cu


In [ ]:
%%writefile CUDA_practice2.cu

#include <stdio.h>

__device__ void hiDeviceFunction(void) // GPU에서만 돌아가며, GPU에서만 Qualify 된다..
{
  printf("Hello! This is in hiDeviceFunciton. \n");
}

__global__ void helloCUDA(void) // Local + GPU
{
  printf("Hello thread %d\t block Index%d\n", threadIdx.x, blockIdx.x); // <<< 에 의해서 만들어지는 threadIdx
  hiDeviceFunction();
}

int main()
{
  helloCUDA<<<2, 4>>>(); // <<< -> block 하나를만들어라, >>> 1개의 Thread를 만들어라.

  //Asynchronous에 의해 Function이 호출 되기 전에 종료가 되어버림. 
  cudaDeviceSynchronize(); //cudaDeviceReset 도 가능

  return 0;
}

Writing CUDA_practice2.cu


In [ ]:
%%writefile CUDA_practice3.cu

#include <stdio.h>

__device__ void hiDeviceFunction(void) // GPU에서만 돌아가며, GPU에서만 Qualify 된다..
{
  printf("Hello! This is in hiDeviceFunciton. \n");
}

__global__ void helloCUDA(void) // Local + GPU
{
  printf("Hello thread %d\t block Index%d\t block Dimension = %d\n", threadIdx.x, blockIdx.x, blockDim.x); // <<< 에 의해서 만들어지는 threadIdx

  // gloabl IDX -> block num -> thread num
  int global_index = blockIdx.x*blockDim.x + threadIdx.x; // Define blockDim, we do not change the number
  printf("Hello unique thread ID = %d\n", global_index);
  //hiDeviceFunction();
}

int main()
{
  helloCUDA<<<2, 4>>>(); // <<< -> block 하나를만들어라, >>> 1개의 Thread를 만들어라.

  //Asynchronous에 의해 Function이 호출 되기 전에 종료가 되어버림. 
  cudaDeviceSynchronize(); //cudaDeviceReset 도 가능

  return 0;
}

Overwriting CUDA_practice3.cu


In [ ]:
%%writefile CUDA.cu

#include <stdio.h>

__global__ void saxpy(int n, float a, float* x, float* y)
{
  int idx = blockIdx.x * blockDim.x + threadIdx.x;

  if (idx < n)
  {
    y[idx] = a*x[idx] + y[idx];
  }
}

int main()
{
  int n = 1024*1024;
  float a = 0.15;
  float x[n];
  float y[n];
  float h_y[n];
  float* d_x;
  float* d_y;

  //initialize
  for(int i=0; i<n; i++) 
  {
    x[n] = i*0.01 + 0.4;
    y[n] = i*0.01 + 0.4;
    h_y[n] = i*0.01 + 0.4;
  }

  cudaMalloc(&d_x, n*sizeof(float));
  cudaMalloc(&d_y, n*sizeof(float));

  cudaMemcpy(d_x, x, n*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, n*sizeof(float), cudaMemcpyHostToDevice);

  int threadPerblock = 256;
  int blocks = n/ 256;

  saxpy <<< blocks, threadPerblock>>>(n,a, d_x, d_y);

  cudaMemcpy(x, d_x, n*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(y, d_y, n*sizeof(float), cudaMemcpyHostToDevice);
 
  //Correct funtion
  for(int i=0; i<n; i++)
  {
    h_y[n] = a*x[i] + h_y[i];
  }

  float diff;
  float errorSum = 0;
  for(int i=0; i<n; i++)
  {
    diff= y[i] - h_y[i];
    errorSum += diff * diff;
  }

  printf("ErrorSum = %f", errorSum);

  cudaFree(d_x);
  cudaFree(d_y);

}


Overwriting CUDA.cu


In [ ]:
!ls

a.out	 CUDA_practice2.cu  CUDA_practice.cu
CUDA.cu  CUDA_practice3.cu  sample_data


In [ ]:
!nvcc CUDA.cu

In [ ]:
!ls

a.out	 CUDA_practice2.cu  CUDA_practice.cu
CUDA.cu  CUDA_practice3.cu  sample_data


In [ ]:
!./a.out